# Practical 4

## Text Classification using Recurrent Neural Network 

Language is naturally composed of sequence data, in the form of characters in words, and words in sentences. Other examples of sequence data include stock prices and  weather data over time. Elements in the data have a relationship with what comes before and what comes after, and this fact requires a different approach.

In this lab exercise, we will learn to use LSTM (an RNN variant) to train a model to classify a piece of text as expressing positive sentiment or negative sentiment.

In [1]:
import os
import shutil
import tensorflow as tf

from datetime import datetime
import tensorflow as tf

### Download the IMDb Dataset
You will use the [Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/). You will train a sentiment classifier model on this dataset.

In [2]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

84125825/84125825 ━━━━━━━━━━━━━━━━━━━━ 1005s 12us/step


FileNotFoundError: [Errno 2] No such file or directory: 'aclImbdb_v1_extracted/aclImdb'

In [6]:
os.listdir("./")

['Practical-4-edit.py',
 'Lesson3_data.zip',
 'Practical-4-edit.ipynb',
 'Practical_5_edit_2024_2.ipynb',
 'Practical-3-edit.ipynb',
 'data.zip',
 '.jukit',
 'p2.md',
 'Practical-6-v2-edit.ipynb',
 'aclImdb_v1_extracted',
 'aclImdb_v1.tar.gz',
 'p4.md',
 '.virtual_documents',
 'Practical-2-edit.py',
 'anki-p1.md',
 'p1.md',
 '.ipynb_checkpoints',
 'Practical-1 (1).ipynb',
 'Practical-2-edit.ipynb',
 'Practical-1.ipynb']

In [10]:
dataset_dir = 'aclImdb_v1_extracted/aclImdb'
os.listdir(dataset_dir)

['imdbEr.txt', 'test', 'imdb.vocab', 'README', 'train']

Take a look at the `train/` directory. It has `pos` and `neg` folders with movie reviews labelled as positive and negative respectively. You will use reviews from `pos` and `neg` folders to train a binary classification model.

In [11]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['urls_unsup.txt',
 'neg',
 'urls_pos.txt',
 'urls_neg.txt',
 'pos',
 'unsupBow.feat',
 'labeledBow.feat']

The `train` directory also has additional folders which should be removed before creating training dataset.

In [12]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

FileNotFoundError: [Errno 2] No such file or directory: 'aclImdb_v1_extracted/aclImdb/train/unsup'

Next, create a `tf.data.Dataset` using `tf.keras.preprocessing.text_dataset_from_directory`. You can read more about this utility from the [api documentation](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text_dataset_from_directory). 

Use the `train` directory to create both train and validation datasets with a split of 20% for validation. Also note that here we use a smaller batch size of 128, as our model now is more complex, and will use up some significant memory, leaving little room for larger batch size.

In [13]:
batch_size = 128
seed = 123
train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb_v1_extracted/aclImdb/train', batch_size=batch_size, validation_split=0.2, 
    subset='training', seed=seed)
val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb_v1_extracted/aclImdb/train', batch_size=batch_size, validation_split=0.2, 
    subset='validation', seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


Take a look at a few movie reviews and their labels `(1: positive, 0: negative)` from the train dataset.

In [14]:
for text_batch, label_batch in train_ds.take(1):
    for i in range(3):
        print(label_batch[i].numpy(), text_batch[i].numpy())

1 b"I have watched this movie well over 100-200 times, and I love it each and every time I watched it. Yes, it can be very corny but it is also very funny and enjoyable. The camp shown in the movie is a real camp that I actually attended for 7 years and is portrayed as camp really is, a great place to spend the summer. Everyone who has ever gone to camp, wanted to go to camp, or has sent a child to camp should see this movie because it'll bring back wonderful memories for you and for your kids."
1 b'This movie is SOOOO funny!!! The acting is WONDERFUL, the Ramones are sexy, the jokes are subtle, and the plot is just what every high schooler dreams of doing to his/her school. I absolutely loved the soundtrack as well as the carefully placed cynicism. If you like monty python, You will love this film. This movie is a tad bit "grease"esk (without all the annoying songs). The songs that are sung are likable; you might even find yourself singing these songs once the movie is through. This m

2025-02-11 22:47:56.712333: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Configure the dataset for performance

These are two important methods you should use when loading data to make sure that I/O does not become blocking.

`.cache()` keeps data in memory after it's loaded off disk. This will ensure the dataset does not become a bottleneck while training your model. If your dataset is too large to fit into memory, you can also use this method to create a performant on-disk cache, which is more efficient to read than many small files.

`.prefetch()` overlaps data preprocessing and model execution while training. 

You can learn more about both methods, as well as how to cache data to disk in the [data performance guide](https://www.tensorflow.org/guide/data_performance).

In [15]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Next, define the dataset preprocessing steps required for your sentiment classification model. Initialize a TextVectorization layer with the desired parameters to vectorize movie reviews. 

TextVectorization layer is a text tokenizer which breaks up the text into words (it is similar to Keras Tokenizer but implemented as a layer). You can read more about TextVectorization layer [here](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/TextVectorization).


In [16]:
# Vocabulary size and number of words in a sequence.
VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 200
# Use the text vectorization layer to normalize, split, and map strings to 
# integers.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE, 
    output_sequence_length=MAX_SEQUENCE_LENGTH
)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

2025-02-11 22:48:03.545529: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [17]:
print(len(vectorize_layer.get_vocabulary()))

10000


## Create a classification model

1. This model can be built as a `tf.keras.Sequential`.

2. The first layer is the vectorization layer, which converts the text to a sequence of token indices.

3. After the vectorization layer is an embedding layer. An embedding layer stores one vector per word. When called, it converts the sequences of word indices to sequences of vectors. These vectors are trainable. After training (on enough data), words with similar meanings often have similar vectors.

4. A recurrent neural network (RNN) processes sequence input by iterating through the elements. RNNs pass the outputs from one timestep to their input on the next timestep. The `tf.keras.layers.Bidirectional` wrapper can also be used with an RNN layer. This propagates the input forward and backwards through the RNN layer and then concatenates the final output. 

5. After the RNN has converted the sequence to a single vector the two `layers.Dense` do some final processing, and convert from this vector representation to a single logit as the classification output. 

In [18]:
EMBEDDING_DIM=128

model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, 
              output_dim=EMBEDDING_DIM, 
              mask_zero=True, 
              name='embedding'),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

Compile and train the model using the `Adam` optimizer and `BinaryCrossentropy` loss. 

In [19]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [20]:
model.fit(
    train_ds, 
    validation_data=val_ds,
    epochs=1)

157/157 ━━━━━━━━━━━━━━━━━━━━ 41s 256ms/step - accuracy: 0.6759 - loss: 0.5556 - val_accuracy: 0.8648 - val_loss: 0.3273


Let's evaluate the model on our test dataset.

In [21]:
test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb_v1_extracted/aclImdb/test', 
    batch_size=128)

Found 25000 files belonging to 2 classes.


In [22]:
model.evaluate(test_ds)

196/196 ━━━━━━━━━━━━━━━━━━━━ 16s 80ms/step - accuracy: 0.8467 - loss: 0.3516


[0.35155218839645386, 0.8466799855232239]

Here we show how we can use get all the individual predictions for the test_ds and use the predictions to plot the confusion_matrix and classification report to allow us to have better insight.

In [23]:
import numpy as np

y_preds = np.array([])
y_labels = np.array([])
count = 0
for texts, labels in test_ds:
    preds = model.predict(texts)
    preds = (preds >= 0.5).reshape(-1)
    y_preds = np.concatenate((y_preds, preds), axis=0)
    y_labels = np.concatenate((y_labels, labels), axis=0)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
4/4 ━━━━━━━━

In [24]:
from sklearn.metrics import classification_report

print(classification_report(y_labels, y_preds))

ModuleNotFoundError: No module named 'sklearn'

Now let us put our model to use

In [ ]:
text = input("Write your review here:")

In [ ]:
pred = model.predict( tf.convert_to_tensor([text]))[0]
if pred >= 0.5: 
    print('positive sentiment')
else:
    print('negative sentiment')

# Stack two or more LSTM layers

Keras recurrent layers have two available modes that are controlled by the `return_sequences` constructor argument:

* If `False` it returns only the last output for each input sequence (a 2D tensor of shape (batch_size, output_features)). This is the default, used in the previous model.

* If `True` the full sequences of successive outputs for each timestep is returned (a 3D tensor of shape `(batch_size, timesteps, output_features)`).


In [ ]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
model.fit(train_ds, 
          epochs=1,
          validation_data=val_ds)

In [ ]:
test_loss, test_acc = model.evaluate(test_ds)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

In [ ]:
sample_text = 'i dozed off during the show.'
predictions = model.predict(tf.convert_to_tensor([sample_text]))
print( 'positive' if predictions >= 0.5 else 'negative')